Imports

In [1]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

import sys
sys.path.append('../../')  # replace with the actual path
import lib.key_param as key_param
import lib.Utils as utils

### Load the Document

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../../sample_files/7cc24bccf34a87a9bc817fe89e320a112a9cec26b9c5db7250e1cef7dc06eff0.pdf")
pages = loader.load_and_split()

Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEnc

In [3]:
chunk1 = pages[2].page_content
print(chunk1[0:100])

3
AdoptedThe European Data Protection Board
Having regard to Article 70 (1 )(e) of the Regulation 20


## Split

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)

### Embeddings

In [12]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [13]:
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=embeddings)

retriever = vectorstore.as_retriever()

### Retrieval and Generation

In [14]:
from langchain.prompts import ChatPromptTemplate

# from langchain import hub
# prompt = hub.pull("usctrojan/in-house-legal")

# Prompt
template_rag = """System Message:

You are a privacy consultant for a marketing company that operate internationally. Provide your legal advice based on the context.

Context: {context}

Question: {question}
"""

prompt_rag = ChatPromptTemplate.from_template(template_rag)

In [15]:
prompt_rag.pretty_print()

================================ Human Message =================================

System Message:

You are a privacy consultant for a marketing company that operate internationally. Provide your legal advice based on the context.

Context: {context}

Question: {question}


### Model Selections

In [24]:
from langchain_community.llms import Ollama

llm = Ollama(model="phi3")

### Post-Processing

In [25]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

### Chain

In [26]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_rag
    | llm
    | StrOutputParser()
)

In [27]:
# 1 (bad) - what is the edpb position in relation to the one-stop shop mechanism?
# 1.2 (good) - What is the EDPB position in relation to the one stop shop mechanism provided by article 52 of the GDPR in the context of the of geographical scope of application of the GDPR (art. 3.2), so called targeting?
#     -- (art. 56 is the correct one)

output_rag = rag_chain.invoke("In the context of the material scope of application of GDPR, what is the edpb position in relation to the one-stop shop mechanism?")

In [29]:
print("--- RAG ---\n")
print(output_rag, 100)

--- RAG ---

As a privacy consultant for your marketing company, based on the provided context regarding the territorial scope of the General Data Protection Regulation (GDPR) and related guidelines by the European Data Protection Board (EDPB), my legal advice is as follows:


The EDPB recognizes the one-stop shop mechanism under Article 25(3) GDPR, which allows a single supervisory authority to be responsible for overseeing an organization's data protection activities across multiple Member States if its core establishment and substantial parts of its activities are located in that particular state. This means:

1. If your company has both controllers and processors operating within the EU, but with a significant presence (core establishment) primarily in one Member State where it also conducts most of its processing operations, you will likely benefit from engaging just one Supervisory Authority to handle all GDPR-related matters related to that central hub.

2. However, if your comp